In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np

In [2]:
data_regression = pd.read_csv("../data/kc_house_data.csv")
data_classification = pd.read_csv("../data/card_transdata.csv")

## Задание 1:

In [3]:
data_classification.info()
data_classification['used_chip'] = data_classification['used_chip'].astype(int)
data_classification['repeat_retailer'] = data_classification['repeat_retailer'].astype(int)
data_classification['used_pin_number'] = data_classification['used_pin_number'].astype(int)
data_classification['online_order'] = data_classification['online_order'].astype(int)
data_classification['fraud'] = data_classification['fraud'].astype(int)
data_classification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   distance_from_home              1000000 non-null  float64
 1   distance_from_last_transaction  1000000 non-null  float64
 2   ratio_to_median_purchase_price  1000000 non-null  float64
 3   repeat_retailer                 1000000 non-null  float64
 4   used_chip                       1000000 non-null  float64
 5   used_pin_number                 1000000 non-null  float64
 6   online_order                    1000000 non-null  float64
 7   fraud                           1000000 non-null  float64
dtypes: float64(8)
memory usage: 61.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          -------

In [4]:
data_regression = data_regression.drop(["date","id"], axis = 1)

In [5]:
data_classification.head(10)

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1,1,0,0,0
1,10.829943,0.175592,1.294219,1,0,0,0,0
2,5.091079,0.805153,0.427715,1,0,0,1,0
3,2.247564,5.600044,0.362663,1,1,0,1,0
4,44.190936,0.566486,2.222767,1,1,0,1,0
5,5.586408,13.261073,0.064768,1,0,0,0,0
6,3.724019,0.956838,0.278465,1,0,0,1,0
7,4.848247,0.320735,1.273050,1,0,1,0,0
8,0.876632,2.503609,1.516999,0,0,0,0,0
9,8.839047,2.970512,2.361683,1,0,0,1,0


In [6]:
data_regression.head(10)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,1225000.0,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,229500.0,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,323000.0,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [7]:
y_regression = data_regression["price"]
X_regression = data_regression.drop(columns = ['price'])
y_classification = data_classification['fraud']
X_classification = data_classification.drop(columns = ['fraud'])

In [8]:
from sklearn.model_selection import train_test_split
X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.25)
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.25)

In [9]:
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(18,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [10]:
model_regression.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1216      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3841 (15.00 KB)
Trainable params: 3841 (15

In [11]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [12]:
model_regression.fit(X_regression_train, y_regression_train, epochs=10)

Epoch 1/10
507/507 [==============================] - 1s 1ms/step - loss: 106122510336.0000
Epoch 2/10
507/507 [==============================] - 1s 1ms/step - loss: 68235812864.0000
Epoch 3/10
507/507 [==============================] - 1s 1ms/step - loss: 70451511296.0000
Epoch 4/10
507/507 [==============================] - 1s 1ms/step - loss: 67147579392.0000
Epoch 5/10
507/507 [==============================] - 1s 1ms/step - loss: 67114266624.0000
Epoch 6/10
507/507 [==============================] - 1s 1ms/step - loss: 68151451648.0000
Epoch 7/10
507/507 [==============================] - 1s 1ms/step - loss: 65453764608.0000
Epoch 8/10
507/507 [==============================] - 1s 1ms/step - loss: 66247409664.0000
Epoch 9/10
507/507 [==============================] - 1s 1ms/step - loss: 64767606784.0000
Epoch 10/10
507/507 [==============================] - 1s 1ms/step - loss: 64301776896.0000


In [13]:
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))

169/169 [==============================] - 0s 738us/step
59865125721.94519


## Классификация

In [14]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(7,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(2, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [15]:
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")

In [16]:
model_classification_1.fit(X_classification_train, y_classification_train, epochs=10)

Epoch 1/10
23438/23438 [==============================] - 31s 1ms/step - loss: 0.0238
Epoch 2/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0070
Epoch 3/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0055
Epoch 4/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0050
Epoch 5/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0043
Epoch 6/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0042
Epoch 7/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0039
Epoch 8/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0035
Epoch 9/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0037
Epoch 10/10
23438/23438 [==============================] - 30s 1ms/step - loss: 0.0032


In [17]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    228149
           1       0.99      0.97      0.98     21851

    accuracy                           1.00    250000
   macro avg       1.00      0.98      0.99    250000
weighted avg       1.00      1.00      1.00    250000

[[228006    143]
 [   726  21125]]


In [24]:
model_classification_1.save('../notebooks/nn.h5', save_format='h5')

## MLP

In [178]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [306]:
class MLP:   
    def __init__(self, input_size ,layersCount ,LayersInfo , learning_rate):
        self.input_size = input_size
        self.hidden_layers = layersCount
        self.neurons = LayersInfo[0]
        print(self.neurons)
        self.activations = list(map(lambda x: self.addActivations[x], LayersInfo[1]))
        self.derivatives = list(map(lambda x: self.addDerivatives[x], LayersInfo[1]))
        self.learning_rate = learning_rate
        self.weights = []
        self.bias = []
        self.loss_arr = [] 

    def _FindWeightsAndBias(self, input, index):
        self.weights.append(np.random.rand(input, self.neurons[index])*0.005)
        self.bias.append(np.random.rand(1, self.neurons[index])*0.005)

    addActivations = {'sigmoid': lambda x: math.round(1 / (1 + np.exp(-x))),
                     'relu': lambda x: np.maximum(0, x),
                     'tanh': lambda x: np.tanh(x),
                     'linear': lambda x: x}
    
    addDerivatives = {'sigmoid': lambda x: math.round(np.exp(-x) / ((1 + np.exp(-x))**2)),
                     'relu': lambda x: np.where(x > 0, 1, 0),
                     'tanh': lambda x: 1 - np.tanh(x)**2,
                     'linear': lambda x: 1}

    def MSE(self, y_test, y_pred):
        f = lambda x,y: pow(x-y, 2) 
        return np.sum(list(map(f, y_test, y_pred)))/len(y_test)

    def _start(self, X, y):
        y = self.y_data
        self.input = []
        self.weights = []
        self.bias = []
        self.layers = []
        self.input.append(X)
        for i in range(self.hidden_layers):
            print(self.input[-1].shape)
            self._FindWeightsAndBias(len(self.input[-1].T), i)

            print(self.weights[-1].shape)
            print(self.bias[-1].shape)
            layer = self.input[-1].dot(self.weights[-1]) + self.bias[-1]

            print(layer.shape, end = '\n\n')
            self.layers.append(layer)
            dense = self.activations[i](layer)
            self.input.append(dense)

        print(self.MSE(y, self.input[-1]))
        self.loss_arr.append(self.MSE(y, self.input[-1]))

        return self.input[-1]

    def _forward(self, X, y):
        y = self.y_data
        self.input = []
        self.layers = []
        self.input.append(X)
        for i in range(self.hidden_layers):
            layer = self.input[i].dot(self.weights[i]) + self.bias[i]
            self.layers.append(layer)
            dense = self.activations[i](layer)
            self.input.append(dense)

        print(self.MSE(y, self.input[-1]))
        self.loss_arr.append(self.MSE(y, self.input[-1]))
        r = lambda x: np.array(map(round, x))
        self.input[-1]=np.array(r(v) for v in self.input[-1])
        return self.input[-1]

    
    def _backward(self, X, y):
        delta_t = (self.input[-1] - y[:, np.newaxis])*np.array(self.derivatives[-1](self.layers[-1]))
        for i in range(1,len(self.weights)-1):
            self.weights[-i]-=self.learning_rate*(self.layers[-i-1].T.dot(delta_t))/len(y)
            self.bias[-i]-=np.sum(self.weights[-i])/len(y)
            delta_t = self.derivatives[-i-1](self.layers[-i-1])*(delta_t.dot(self.weights[-i].T))
            
    def train(self, X, y, epochs):
        self.y_data = y
        output = self._start(X,y)
        for epoch in range(epochs):
            self._backward(X, y)
            output = self._forward(X,y)
        plt.plot(self.loss_arr)
        plt.show()
        

    def predict(self, X):
        return self._forward(X, [])


In [307]:
data_reg = data_regression.sample(5000)
data_classif = data_classification.sample(5000)

from imblearn.under_sampling import NearMiss

y_regression = np.array(data_reg["price"])
X_regression = np.array(data_reg.drop(columns = ['price']))
y_classification = np.array(data_classif['fraud'])
X_classification = np.array(data_classif.drop(columns = ['fraud']))

nm = NearMiss()
X_nm_train, y_nm_train = nm.fit_resample(X_classification, y_classification)

In [308]:
testMLP = MLP(18, 4, [[64, 32, 16,1],['relu','linear','relu','linear']], 0.005)

[64, 32, 16, 1]


In [309]:
testMLP.train(X_regression, y_regression, 10)

(5000, 18)
(18, 64)
(1, 64)
(5000, 64)

(5000, 64)
(64, 32)
(1, 32)
(5000, 32)

(5000, 32)
(32, 16)
(1, 16)
(5000, 16)

(5000, 16)
(16, 1)
(1, 1)
(5000, 1)

440830107494.58606
4.716792278207831e+35


TypeError: unsupported operand type(s) for -: 'generator' and 'float'

In [310]:
y_pred = testMLP.predict(X_regression)

4.716792278207831e+35


In [311]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [312]:
mean_squared_error(y_regression, y_pred)

TypeError: Singleton array array(<generator object MLP._forward.<locals>.<genexpr> at 0x0000026D98AFE3C0>,
      dtype=object) cannot be considered a valid collection.

In [313]:
mean_absolute_percentage_error(y_regression, y_pred)

TypeError: Singleton array array(<generator object MLP._forward.<locals>.<genexpr> at 0x0000026D98AFE3C0>,
      dtype=object) cannot be considered a valid collection.

In [314]:
y_regression

array([684000., 619100., 530000., ..., 390000., 770000., 600000.])

In [315]:
y_pred

array(<generator object MLP._forward.<locals>.<genexpr> at 0x0000026D98AFE3C0>,
      dtype=object)

In [316]:
testMLPclass = MLP(7, 6, [[64, 128, 64, 32, 2, 1],['relu','relu' ,'relu' ,'relu','relu','sigmoid']], 0.0005)

[64, 128, 64, 32, 2, 1]


In [317]:
testMLPclass.train(X_nm_train, y_nm_train, 50)

(862, 7)
(7, 64)
(1, 64)
(862, 64)

(862, 64)
(64, 128)
(1, 128)
(862, 128)

(862, 128)
(128, 64)
(1, 64)
(862, 64)

(862, 64)
(64, 32)
(1, 32)
(862, 32)

(862, 32)
(32, 2)
(1, 2)
(862, 2)

(862, 2)
(2, 1)
(1, 1)
(862, 1)



NameError: name 'math' is not defined

In [318]:
y_pred = testMLPclass.predict(X_nm_train)

NameError: name 'math' is not defined

In [319]:
mean_squared_error(y_nm_train, y_pred)

TypeError: Singleton array array(<generator object MLP._forward.<locals>.<genexpr> at 0x0000026D98AFE3C0>,
      dtype=object) cannot be considered a valid collection.

In [320]:
np.unique(y_nm_train)

array([0, 1])

In [321]:
np.unique(y_pred)

array([<generator object MLP._forward.<locals>.<genexpr> at 0x0000026D98AFE3C0>],
      dtype=object)